# Lesson 4: Persistence and Streaming

## What are Persistence and Streaming used for?

### **Persistence (Checkpointing)**

1. **Multi-turn conversations** - Agent remembers previous messages
   - Example: "What's the weather in SF?" → "What about LA?" → "Which is warmer?" 
   - The agent remembers SF and LA context from earlier questions

2. **Multiple users/sessions** - Different `thread_id` keeps conversations separate
   - User A's chat history stays separate from User B's
   - Essential for multi-user applications

3. **Resume after interruption** - Save state, restart later
   - Long-running tasks can be paused and continued
   - Prevents loss of progress if system crashes

4. **Human-in-the-loop** - Stop for approval, then continue
   - Agent asks "Should I delete this file?" → waits for human decision → continues
   - Critical for safety in automated systems

### **Streaming**

1. **Real-time UI updates** - Like ChatGPT's typing effect
   - Show tokens as they generate instead of waiting for complete response
   - Users see progress immediately

2. **Progress monitoring** - See what agent is doing
   - "Searching web..." → "Analyzing results..." → "Generating answer..."
   - Transparency in agent workflow

3. **Better UX** - Users don't stare at blank screen
   - Especially important for slow LLM responses (30+ seconds)
   - Reduces perceived wait time

4. **Debug/logging** - Track agent workflow in real-time
   - See each tool call, LLM invocation, and state update as it happens
   - Easier troubleshooting during development

### **Real-world Use Cases**

- **Customer service chatbots** - Persistence for conversation context across messages
- **Code assistants** - Streaming for real-time feedback during multi-step code generation
- **Research agents** - Both: save progress while streaming results as they're found
- **Data processing pipelines** - Checkpoint progress, stream status updates
- **Interactive tutoring systems** - Remember student history, show explanations incrementally

In [2]:
# Libries
from dotenv import load_dotenv
_ = load_dotenv()
load_dotenv('/home/jovyan/work/.env')  # Docker mounted directory


True

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver


In [4]:
tool = TavilySearchResults(max_results=2)

/tmp/ipykernel_222/4289725543.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
import sqlite3
conn = sqlite3.connect(":memory:", check_same_thread=False)
memory = SqliteSaver(conn)

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [12]:
thread = {"configurable": {"thread_id": "1"}}

In [13]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ad98c18a04', 'id': 'chatcmpl-DABMvpROIPGdv1S9VPtThew5gLFVU', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6adc-5e2d-7fc3-bf6e-315ebeac3139-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_vaTIun02txERBQJM6BA745GG', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a

In [14]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 910, 'total_tokens': 932, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DABNKWGLCvmi8PQVqgRMrv4Y2qkii', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6adc-bd9c-7962-af7a-e2df8690432e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_UXaYcRLpc9PVvhLA2TUquNTC', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 910, 'output_tokens': 22, 'total_tokens': 932, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de

In [15]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Los Angeles is currently warmer than San Francisco. Los Angeles has a temperature of 11°C, while San Francisco has a temperature of 9°C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1943, 'total_tokens': 1974, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1920}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DABNiOm3mR7PCwYNMbsf6UvIQ5tvM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6add-1a50-76d3-a277-97e45eae2541-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1943, 'output_tokens': 31, 'total_tokens': 1974, 'input_token_details': {'audio': 0, 'cache_read': 1920}, 'output_token_details': {'a

In [16]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please clarify what you are comparing in terms of warmth? Are you asking about the weather in different locations, the temperature of substances, or something else? Let me know so I can assist you effectively!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 149, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DABNsQDj1Q8fv7vqksnIpmPZbY5kP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6add-45e0-7f40-8d55-b913c4187cf2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 149, 'output_tokens': 43, 'total_tokens': 192, 'input_t

## Streaming tokens

In [20]:
!pip install -q aiosqlite

In [30]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
import aiosqlite

# Create the async memory properly - must be awaited in async context
async_conn = await aiosqlite.connect(":memory:")
async_memory = AsyncSqliteSaver(async_conn)
abot = Agent(model, [tool], system=prompt, checkpointer=async_memory)

In [26]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_GZSh16C5ppQireER38RPCpmH', 'type': 'tool_call'}
Back to the model!
The| current| weather| in| San| Francisco| is| cool| with| a| temperature| of| |9|°C|,| which| feels| like| |6|°C|.| The| wind| is| coming| from| the| west|-s|outh|west| at| a| speed| of| |23| km|/h|,| creating| a| moderate| breeze|.| There| is| a| |60|%| chance| of| rain| with| |100|%| cloud| cover|.| The| humidity| is| at| |82|%,| and| the| atmospheric| pressure| is| |100|7| mb|.| The| forecast| for| the| afternoon| remains| similar|,| and| for| tonight|,| the| temperature| is| expected| to| drop| to| |7|°C| with| partly| cloudy| conditions|.|

## Additional Streaming Example: Real-time Research Assistant

This example shows how streaming provides transparency in a multi-step research workflow.

In [31]:
# Example: Track agent workflow with streaming
# This shows what the agent is doing at each step

messages = [HumanMessage(content="Compare the GDP of California and Texas. Which is higher?")]
thread = {"configurable": {"thread_id": "5"}}

print("🔍 STARTING RESEARCH AGENT\n")
print("=" * 60)

async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    
    # Show when agent starts thinking
    if kind == "on_chain_start" and event.get("name") == "llm":
        print("\n🤔 Agent is thinking...")
    
    # Show when agent calls a tool
    if kind == "on_tool_start":
        tool_name = event.get("name", "unknown")
        print(f"\n🔧 Calling tool: {tool_name}")
        print(f"   Input: {event['data'].get('input', {})}")
    
    # Show when tool returns result
    if kind == "on_tool_end":
        print(f"✅ Tool completed")
    
    # Stream the actual response content
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)

print("\n" + "=" * 60)
print("✅ RESEARCH COMPLETE")

🔍 STARTING RESEARCH AGENT


🤔 Agent is thinking...
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of California 2023'}, 'id': 'call_6q4qx7OkL1vqbmFxlvUoZlE8', 'type': 'tool_call'}

🔧 Calling tool: tavily_search_results_json
   Input: {'query': 'GDP of California 2023'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Texas 2023'}, 'id': 'call_LXbNfitEinQ8by2I4xtFT1nK', 'type': 'tool_call'}✅ Tool completed


🔧 Calling tool: tavily_search_results_json
   Input: {'query': 'GDP of Texas 2023'}
Back to the model!✅ Tool completed


🤔 Agent is thinking...
As of 2023, California's GDP was about $3.9 trillion, whereas Texas's GDP was significantly lower at approximately $2.769 trillion. Therefore, California has a higher GDP than Texas.
✅ RESEARCH COMPLETE


**What this demonstrates:**
- **Transparency**: User sees each step (thinking, tool calls, results)
- **Progress feedback**: No blank screen - shows agent is working
- **Real-time streaming**: Final answer appears token-by-token
- **Better debugging**: Can see exactly where agent might get stuck

**Output format:**
```
🔍 STARTING RESEARCH AGENT
============================================================

🤔 Agent is thinking...

🔧 Calling tool: tavily_search_results_json
   Input: {'query': 'California GDP 2024'}
✅ Tool completed

🔧 Calling tool: tavily_search_results_json
   Input: {'query': 'Texas GDP 2024'}
✅ Tool completed

🤔 Agent is thinking...

Based on the search results, California has a GDP of...
[streaming tokens appear here in real-time]
============================================================
✅ RESEARCH COMPLETE
```

## Comparison: Non-Streaming vs Streaming

Let's see the difference more clearly.

In [36]:
import time

# Use sync memory for non-streaming example
sync_abot = Agent(model, [tool], system=prompt, checkpointer=memory)

# NON-STREAMING: You wait, then see everything at once
print("❌ NON-STREAMING EXAMPLE")
print("=" * 60)
print("⏳ Waiting... (user sees nothing)")

messages = [HumanMessage(content="What's the todays date in Singapore?")]
thread = {"configurable": {"thread_id": "6"}}

start = time.time()
result = sync_abot.graph.invoke({"messages": messages}, thread)
elapsed = time.time() - start

print(f"\n💥 BOOM! Everything appears after {elapsed:.1f}s:")
print(result['messages'][-1].content)
print("=" * 60)

❌ NON-STREAMING EXAMPLE
⏳ Waiting... (user sees nothing)
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current date in Singapore'}, 'id': 'call_cBIDdWxH2ODa3IGPs5CWjmfH', 'type': 'tool_call'}
Back to the model!

💥 BOOM! Everything appears after 4.1s:
Today's date in Singapore is February 17, 2026.


In [37]:
# STREAMING: You see progress immediately
print("\n✅ STREAMING EXAMPLE")
print("=" * 60)
print("👀 User sees everything as it happens:\n")

messages = [HumanMessage(content="What's the todays date in Singapore?")]
thread = {"configurable": {"thread_id": "7"}}

start = time.time()
token_count = 0

async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    
    if kind == "on_tool_start":
        print(f"🔧 Tool: {event.get('name', 'unknown')}")
    
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)
            token_count += 1
            time.sleep(0.05)  # Simulate visible delay to show streaming

elapsed = time.time() - start
print(f"\n\n💡 Streamed {token_count} tokens over {elapsed:.1f}s")
print("=" * 60)


✅ STREAMING EXAMPLE
👀 User sees everything as it happens:

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current date in Singapore'}, 'id': 'call_Ydte8v4VH2D48t7eLA9wmrgn', 'type': 'tool_call'}
🔧 Tool: tavily_search_results_json
Back to the model!
The current date in Singapore is Tuesday, February 17, 2026.

💡 Streamed 16 tokens over 5.8s


### Key Differences:

**Non-Streaming (`.invoke()`):**
- User waits in silence ⏳
- Nothing appears until complete
- All text arrives at once 💥
- Poor UX for slow operations

**Streaming (`.astream_events()`):**
- Immediate feedback 👀
- Tokens appear one by one
- User sees progress in real-time ⚡
- Feels faster even if same duration

**In production apps:**
- Streaming = ChatGPT-style typing effect
- Non-streaming = Spinner, then sudden response
- Streaming reduces perceived wait time by 30-50%

## 今日学习总结 (Today's Learning Summary)

### 🎯 核心概念 (Core Concepts)

#### 1️⃣ **Lesson 1: 从零构建 ReAct Agent**
- **ReAct 模式**: Thought → Action → Observation 循环
- **手动 Action 解析**: 使用正则表达式 `action_re = re.compile('^Action: (\w+): (.*)$')` 提取动作
- **工具集成**: 实现 `calculate()` 和 `average_dog_weight()` 自定义函数
- **上下文管理**: 学习监控 token 使用量，防止超出上下文窗口
- **关键学习**: 理解 Agent 的基本工作原理，完全手动控制流程

#### 2️⃣ **Lesson 2: LangGraph 组件**
- **StateGraph**: 用图结构定义 Agent 工作流
  - **Nodes**: `llm` (思考) 和 `action` (执行工具)
  - **Edges**: 条件边 `exists_action()` 决定是否需要调用工具
- **AgentState**: `TypedDict` + `Annotated` + `operator.add` 管理消息历史
- **工具绑定**: `model.bind_tools(tools)` 让 LLM 知道可用工具
- **本地 Ollama 集成**: 
  - 使用 `base_url="http://host.docker.internal:11434/v1"`
  - 发现 `qwen2.5-coder:14b` 不擅长 tool calling
  - 切换到 `qwen2.5:14b` 获得更好效果
- **关键学习**: 从手动编码到声明式图结构，大幅简化复杂工作流

#### 3️⃣ **Lesson 3: Agentic Search**
- **对比搜索方式**:
  - **传统搜索**: DuckDuckGo → 手动爬取 → BeautifulSoup 解析 → 冗长结果
  - **Agentic 搜索**: Tavily API → 智能筛选 → 结构化返回
- **Tavily vs DuckDuckGo**:
  - Tavily: 付费，质量高，返回精炼内容
  - DuckDuckGo: 免费，但结果混乱，需要额外处理
- **关键学习**: 好的搜索工具对 Agent 性能至关重要

#### 4️⃣ **Lesson 4: 持久化与流式输出**
- **Persistence (持久化)**:
  - `SqliteSaver` 保存对话状态
  - `thread_id` 区分不同用户会话
  - 支持多轮对话记忆（SF → LA → "Which is warmer?"）
  - 用途: 客服机器人、长任务断点续传、人工审核流程
  
- **Streaming (流式输出)**:
  - `.invoke()`: 同步，等待完成后一次性返回 ❌
  - `.astream_events()`: 异步，逐 token 返回 ✅
  - `AsyncSqliteSaver` 用于异步场景
  - 用途: ChatGPT 式打字效果，实时进度显示，降低感知等待时间 30-50%

---

### 🛠️ 技术栈总结

**环境配置**:
- Docker + Jupyter + VS Code Remote Kernel
- 本地 Ollama (M5 24GB RAM)
- `.env` + `.gitignore` 保护 API 密钥

**关键库**:
- `langgraph`: StateGraph 构建工作流
- `langchain`: ChatOpenAI, 消息类型, 工具封装
- `tavily`: 网络搜索 API
- `sqlite3`: 持久化存储
- `aiosqlite`: 异步数据库操作

**模型选择**:
- ❌ `qwen2.5-coder:14b`: 代码优秀，但 tool calling 差
- ✅ `qwen2.5:14b`: 通用模型，tool calling 效果好
- 🏆 `gpt-4o`: 最强性能，完整完成多步推理

---

### 💡 实践经验

1. **API 密钥安全**: 
   - 使用 `.env` 文件
   - 添加到 `.gitignore`
   - Docker 需要将 `.env` 复制到挂载目录

2. **模型选型**:
   - 代码模型（coder variant）不适合 Agent 工作流
   - 本地模型在多步推理时容易中途停止
   - GPT-4o 能完整完成复杂的多步任务

3. **输出优化**:
   - 创建 `print_result()` 美化输出
   - 添加 truncation 处理长文本（max_len=500）
   - 使用 emoji 提升可读性

4. **性能对比**:
   - Tavily 搜索质量 >> DuckDuckGo
   - Streaming 用户体验 >> Non-streaming
   - 本地模型速度快，但准确性不如 GPT-4o

---

### 🚀 下一步建议

1. **练习 LangGraph 高级功能**:
   - 更复杂的条件边
   - 子图 (subgraphs)
   - 循环与中断点

2. **优化本地模型**:
   - 尝试 `mistral-small` 或 `llama3.1:70b`（如果内存足够）
   - 调整 system prompt 提升 tool calling 成功率
   - 实验温度和 top_p 参数

3. **实战项目**:
   - 构建个人研究助手（persistence + streaming）
   - 多轮对话客服机器人
   - 代码生成与执行 Agent

4. **性能监控**:
   - 添加 token 使用统计
   - 记录工具调用成功率
   - 测量响应时间与准确性

---

### 📊 今日成果

✅ 完成 4 个完整课程  
✅ 掌握 ReAct 模式与 LangGraph 架构  
✅ 成功集成本地 Ollama 模型  
✅ 理解 Persistence 与 Streaming 的实际应用  
✅ 建立完整的 Agent 开发环境  
✅ 学会调试和优化 Agent 输出  

**总学习时长**: 约 4-6 小时  
**代码行数**: ~500+ lines  
**运行测试**: 20+ 次 Agent 调用